<a href="https://colab.research.google.com/github/finiteautomata/hatespeech-classification/blob/main/notebooks/Hatespeech_Colab_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ahora, a montar Drive

In [3]:
repo_dir = "/content/drive/Shareddrives/Hate\ Speech/hatespeech-classification"

In [2]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  31195      0 --:--:-- --:--:-- --:--:-- 31006
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
     |████████████████████████████████| 61kB 2.9MB/s 
Uninstalling torch-1.8.1+cu101:
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
Done updating TPU runtime
  Successfully uninstalled torch-1.8.1+cu101
Uninstalling torchvision-0.9.1+cu101:
  Successfully uninstalled torchvision-0.9.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp37-cp37m-linux_x86_64.whl...
- [1 files][132.2 MiB/

In [3]:
from google.colab import drive

drive.mount("/content/drive/")

Mounted at /content/drive/


Al toque perrito

In [4]:
!cd $repo_dir && git pull
!cd $repo_dir && pip install --user -e . 
!pip install emoji pandarallel absl-py

Already up to date.
Obtaining file:///content/drive/Shareddrives/Hate%20Speech/hatespeech-classification
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 194kB 4.7MB/s 
     |████████████████████████████████| 2.1MB 6.8MB/s 
     |████████████████████████████████| 245kB 30.9MB/s 
     |████████████████████████████████| 112kB 24.4MB/s 
     |████████████████████████████████| 901kB 22.1MB/s 
     |████████████████████████████████| 3.3MB 38.0MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, us

### NOTA IMPORTANTE

Reiniciar notebook después de correr esto

In [1]:
import os
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import sys
import fire
import torch
import transformers
from hatedetection import load_datasets
from hatedetection.training import (
    train_hatespeech_classifier, tokenize, load_hatespeech_model_and_tokenizer
)


model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
context = 'title'

"""
Todo esto es para evitar que la memoria salte por los aires por usar 'fork' en XLA
Ver: https://pytorch.org/xla/release/1.8/index.html#torch_xla.distributed.xla_multiprocessing.MpModelWrapper

"""
print("Cargando modelo")

model, tokenizer = load_hatespeech_model_and_tokenizer(model_name, context)
WRAPPED_MODEL = xmp.MpModelWrapper(model)

def train_model(
    model, tokenizer, output_path, train_path=None, test_path=None, context='none',
    batch_size=32, eval_batch_size=16, max_length=None,
    epochs=10, warmup_proportion=0.1,
    ):
    """
    """ 
    print("*"*80)
    print("Training hate speech classifier")

    allowed_contexts = {'none', 'title', 'body'}

    if context not in allowed_contexts:
        print("")
        sys.exit(1)

    print(f"Uses context: {context}")
    print("*"*80, end="\n"*3)

    print("Loading datasets... ", end="")

    add_body = True if context == 'body' else False
    train_dataset, dev_dataset, test_dataset = load_datasets(train_path, test_path, add_body=add_body)

    print("Done")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("")
    print("Loading model and tokenizer... ", end="")
    
    my_tokenize = lambda batch: tokenize(tokenizer, batch, context=context)
    print("Done")


    trainer, evaluation = train_hatespeech_classifier(
        model, my_tokenize, train_dataset=train_dataset, dev_dataset=dev_dataset,
        batch_size=batch_size, eval_batch_size=eval_batch_size, max_length=max_length,
        epochs=epochs, warmup_proportion=warmup_proportion,
    )

    print("\n*3", f"Evaluation: {evaluation}")


    print("\n"*3, "Saving...")
    trainer.save_model(output_path)
    tokenizer.save_pretrained(output_path)

    print(f"Models saved at {output_path}")



INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Cargando modelo


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

In [ ]:
import os
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

repo_dir = "/content/drive/Shareddrives/Hate\ Speech/hatespeech-classification"
output_path = os.path.join(repo_dir, "models", "bert-contextualized-hate-speech-es/")

def _mp_fn(index):
    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)


    train_model(model, tokenizer, output_path=output_path, context='title', batch_size=32, eval_batch_size=8)

xmp.spawn(_mp_fn, start_method="fork", )

********************************************************************************
Training hate speech classifier
Uses context: title
********************************************************************************


Loading datasets... Done

Loading model and tokenizer... Done
Tokenizing and formatting datasets...


********************************************************************************
Training hate speech classifier
Uses context: title
********************************************************************************


Loading datasets... Done

Loading model and tokenizer... Done
Tokenizing and formatting datasets...


********************************************************************************
Training hate speech classifier
Uses context: title
********************************************************************************


Loading datasets... Done

Loading model and tokenizer... Done
Tokenizing and formatting datasets...


********************************************************************************
Training hate speech classifier
Uses context: title
********************************************************************************


Loading datasets... Done

Loading model and tokenizer... Done
Tokenizing and formatting datasets...


********************************************************************************
Training hate speech classifier
Uses context: title
********************************************************************************





********************************************************************************
Training hate speech classifier
Uses context: title
********************************************************************************


Loading datasets... Done

Loading model and tokenizer... Done
Tokenizing and formatting datasets...


********************************************************************************
Training hate speech classifier
Uses context: title
********************************************************************************


Loading datasets... Done

Loading model and tokenizer... Done
Tokenizing and formatting datasets...
